In [1]:
import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt

import numpy as np
import pandas as pd
import seaborn as sns
import env

In [2]:
url = f'mysql+pymysql://{env.user}:{env.password}@{env.host}/curriculum_logs'
query = '''
Select * from logs
left join cohorts on logs.cohort_id = cohorts.id
ORDER BY date ASC, time ASC;
'''
df = pd.read_sql(query, url)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 900223 entries, 0 to 900222
Data columns (total 15 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   date        900223 non-null  object 
 1   time        900223 non-null  object 
 2   path        900222 non-null  object 
 3   user_id     900223 non-null  int64  
 4   cohort_id   847330 non-null  float64
 5   ip          900223 non-null  object 
 6   id          847330 non-null  float64
 7   name        847330 non-null  object 
 8   slack       847330 non-null  object 
 9   start_date  847330 non-null  object 
 10  end_date    847330 non-null  object 
 11  created_at  847330 non-null  object 
 12  updated_at  847330 non-null  object 
 13  deleted_at  0 non-null       object 
 14  program_id  847330 non-null  float64
dtypes: float64(3), int64(1), object(11)
memory usage: 103.0+ MB


In [7]:
df.isnull().sum()

date               0
time               0
path               1
user_id            0
cohort_id      52893
ip                 0
id             52893
name           52893
slack          52893
start_date     52893
end_date       52893
created_at     52893
updated_at     52893
deleted_at    900223
program_id     52893
dtype: int64

In [8]:
df.user_id.value_counts()

11     17913
64     16347
53     12329
314     7783
1       7404
       ...  
66         1
163        1
918        1
212        1
952        1
Name: user_id, Length: 981, dtype: int64

In [9]:
(df.isnull().sum()/df.shape[0]*100)[:] # Percentage of nulls in each column

date            0.000000
time            0.000000
path            0.000111
user_id         0.000000
cohort_id       5.875544
ip              0.000000
id              5.875544
name            5.875544
slack           5.875544
start_date      5.875544
end_date        5.875544
created_at      5.875544
updated_at      5.875544
deleted_at    100.000000
program_id      5.875544
dtype: float64

In [11]:
df[df.program_id.isnull()]

,date,time,path,user_id,cohort_id,ip,id,name,slack,start_date,end_date,created_at,updated_at,deleted_at,program_id
411,2018-01-26,16:46:16,/,48,NaN,97.105.19.61,NaN,None,None,None,None,None,None,None,NaN
412,2018-01-26,16:46:24,spring/extra-features/form-validation,48,NaN,97.105.19.61,NaN,None,None,None,None,None,None,None,NaN
425,2018-01-26,17:54:24,/,48,NaN,97.105.19.61,NaN,None,None,None,None,None,None,None,NaN
435,2018-01-26,18:32:03,/,48,NaN,97.105.19.61,NaN,None,None,None,None,None,None,None,NaN
436,2018-01-26,18:32:17,mysql/relationships/joins,48,NaN,97.105.19.61,NaN,None,None,None,None,None,None,None,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
899897,2021-04-21,12:49:00,javascript-ii,717,NaN,136.50.102.126,NaN,None,None,None,None,None,None,None,NaN
899898,2021-04-21,12:49:02,javascript-ii/es6,717,NaN,136.50.102.126,NaN,None,None,None,None,None,None,None,NaN
899899,2021-04-21,12:51:27,javascript-ii/map-filter-reduce,717,NaN,136.50.102.126,NaN,None,None,None,None,None,None,None,NaN
899900,2021-04-21,12:52:37,javascript-ii/promises,717,NaN,136.50.102.126,NaN,None,None,None,None,None,None,None,NaN


In [13]:
df[df.updated_at.isnull()].user_id.value_counts()

354    2965
736    2358
363    2248
716    2136
368    2085
       ... 
644       6
663       4
62        4
89        3
176       3
Name: user_id, Length: 78, dtype: int64

In [14]:
df[df.path.isnull()]

,date,time,path,user_id,cohort_id,ip,id,name,slack,start_date,end_date,created_at,updated_at,deleted_at,program_id
506305,2020-04-08,09:25:18,None,586,55.0,72.177.240.51,55.0,Curie,#curie,2020-02-03,2020-07-07,2020-02-03 19:31:51,2020-02-03 19:31:51,None,3.0


In [19]:
df[df.created_at == df.updated_at]

,date,time,path,user_id,cohort_id,ip,id,name,slack,start_date,end_date,created_at,updated_at,deleted_at,program_id
0,2018-01-26,09:55:03,/,1,8.0,97.105.19.61,8.0,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,None,1.0
1,2018-01-26,09:56:02,java-ii,1,8.0,97.105.19.61,8.0,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,None,1.0
2,2018-01-26,09:56:05,java-ii/object-oriented-programming,1,8.0,97.105.19.61,8.0,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,None,1.0
3,2018-01-26,09:56:06,slides/object_oriented_programming,1,8.0,97.105.19.61,8.0,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,None,1.0
4,2018-01-26,09:56:24,javascript-i/conditionals,2,22.0,97.105.19.61,22.0,Teddy,#teddy,2018-01-08,2018-05-17,2018-01-08 13:59:10,2018-01-08 13:59:10,None,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
900218,2021-04-21,16:41:51,jquery/personal-site,64,28.0,71.150.217.33,28.0,Staff,#,2014-02-04,2014-02-04,2018-12-06 17:04:19,2018-12-06 17:04:19,None,2.0
900219,2021-04-21,16:42:02,jquery/mapbox-api,64,28.0,71.150.217.33,28.0,Staff,#,2014-02-04,2014-02-04,2018-12-06 17:04:19,2018-12-06 17:04:19,None,2.0
900220,2021-04-21,16:42:09,jquery/ajax/weather-map,64,28.0,71.150.217.33,28.0,Staff,#,2014-02-04,2014-02-04,2018-12-06 17:04:19,2018-12-06 17:04:19,None,2.0
900221,2021-04-21,16:44:37,anomaly-detection/discrete-probabilistic-methods,744,28.0,24.160.137.86,28.0,Staff,#,2014-02-04,2014-02-04,2018-12-06 17:04:19,2018-12-06 17:04:19,None,2.0


In [16]:
df.updated_at.value_counts()

2018-12-06 17:04:19    84031
2019-07-15 16:57:21    40730
2019-01-20 23:18:57    38096
2020-09-21 18:06:27    37109
2020-01-13 21:17:08    36902
2018-05-25 22:25:57    35636
2020-03-23 17:52:16    33844
2020-07-29 18:41:13    33568
2019-09-16 13:07:04    32888
2020-07-13 18:32:19    32015
2018-01-08 13:59:10    30926
2020-05-26 19:22:44    29855
2019-05-28 18:41:05    29356
2018-03-05 14:22:11    28534
2019-11-04 18:27:07    28033
2018-09-17 19:09:51    27749
2019-08-20 14:38:55    26538
2018-07-23 15:02:25    25586
2019-03-18 20:35:06    25359
2020-11-02 20:43:58    23691
2020-02-03 19:31:51    21582
2018-11-05 15:26:37    20743
2020-09-30 15:54:46    17713
2020-12-07 16:58:43    16623
2021-01-20 21:31:11    16397
2016-06-14 19:52:26    14775
2020-12-07 15:20:18    14715
2016-07-18 19:06:27     9587
2021-03-15 18:18:20     8562
2017-09-27 20:22:41     7444
2021-03-15 19:57:09     7276
2017-02-06 17:49:10     4954
2017-03-28 00:33:12     2158
2021-04-12 18:07:21     1672
2017-06-05 20:

In [20]:
df.program_id.value_counts()

2.0    713365
3.0    103412
1.0     30548
4.0         5
Name: program_id, dtype: int64

In [26]:
df[df.name== "Staff"].user_id.value_counts()

11     15178
64     12530
428     5819
1       5787
248     5027
314     4617
53      4132
545     3528
211     3162
581     2961
546     2585
514     2073
315     2042
404     1668
816     1527
742     1507
480     1256
146     1216
521     1088
430      981
744      651
951      583
893      402
572      390
37       374
502      357
618      318
397      305
630      253
41       204
257      160
308      151
513      132
854      131
312      131
738      128
653      117
953       85
539       84
40        66
620       58
370       54
813       49
855       47
745       46
894       29
148       26
461       11
980        3
652        1
592        1
Name: user_id, dtype: int64

#### Cleaning the Data
- following the merging of the tables there are some simple cleaning steps we can take. deleted_at column is 100% null, can be removed. updated_at and created_at are duplicate columns, updated_at can be removed. Slack col is duplicate of name, drop it. id and cohort_id are duplicates, drop id

In [ ]:
def clean_curriculum(df):
    #drop unneeded columns
    df = df.drop(['deleted_at'], axis=1)
    df = df.drop(['created_at'], axis=1)
    df = df.drop(['slack'], axis=1)
    df = df.drop(['id'], axis=1)
    
    #
    
    
    #drop remaining nulls
    df = df.dropna()
    
    return df

In [ ]:
def clean_lending(df):
    #drop unneeded columns
    df = df.drop(['emp_title'], axis=1)
    df = df.drop(['emp_length'], axis=1)
    df = df.drop(['id'], axis=1)
    
    #impute
    df['mort_acc'].fillna(2.0, inplace = True)
    
    #drop remaining amount of nulls
    #df = df.dropna()
    
    return df    